In [1]:
%pip install --quiet git+https://github.com/mozilla-ai/osm-ai-helper.git
%pip install --quiet ultralytics

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from osm_ai_helper.download_osm import download_osm

In [ ]:
TRAIN_AREA = "Galicia"
TEST_AREA = "Asturias"
SELECTOR = "leisure=swimming_pool"
DISCARD = {"location": "indoor"}
CLASS_NAME = "swimming_pool"

In [ ]:
download_osm(
    area=TRAIN_AREA,
    output_dir="datasets",
    selector=SELECTOR,
    discard=DISCARD,
)

2025-02-12 12:16:07.812 | INFO     | osm_ai_helper.download_osm:download_osm:39 - Downloading osm data for Ponteareas
2025-02-12 12:16:09.268 | INFO     | osm_ai_helper.download_osm:download_osm:47 - Writing 103 elements to datasets/Ponteareas.json
2025-02-12 12:16:09.274 | SUCCESS  | osm_ai_helper.download_osm:download_osm:49 - Done!


In [ ]:
download_osm(
    area=TEST_AREA,
    output_dir="datasets",
    selector=SELECTOR,
    discard=DISCARD,
)

2025-02-12 12:15:18.827 | INFO     | osm_ai_helper.download_osm:download_osm:39 - Downloading osm data for Mondariz
2025-02-12 12:15:19.803 | INFO     | osm_ai_helper.download_osm:download_osm:47 - Writing 9 elements to datasets/Mondariz.json
2025-02-12 12:15:19.806 | SUCCESS  | osm_ai_helper.download_osm:download_osm:49 - Done!


In [6]:
from google.colab import userdata

from osm_ai_helper.group_elements_and_download_tiles import (
    group_elements_and_download_tiles,
)

In [9]:
group_elements_and_download_tiles(
    f"datasets/{TRAIN_AREA}.json",
    f"datasets/{TRAIN_AREA}",
    userdata.get("MAPBOX_TOKEN"),
)

2025-02-12 12:16:12.914 | INFO     | osm_ai_helper.group_elements_and_download_tiles:group_elements_and_download_tiles:35 - Grouping elements by tile
2025-02-12 12:16:12.935 | INFO     | osm_ai_helper.group_elements_and_download_tiles:group_elements_and_download_tiles:40 - Downloading tiles and writing annotation
2025-02-12 12:16:12.939 | INFO     | osm_ai_helper.group_elements_and_download_tiles:group_elements_and_download_tiles:43 - Processed 0/70 tiles
2025-02-12 12:16:15.917 | INFO     | osm_ai_helper.group_elements_and_download_tiles:group_elements_and_download_tiles:43 - Processed 50/70 tiles


In [10]:
group_elements_and_download_tiles(
    f"datasets/{TEST_AREA}.json", f"datasets/{TEST_AREA}", userdata.get("MAPBOX_TOKEN")
)

2025-02-12 12:16:17.930 | INFO     | osm_ai_helper.group_elements_and_download_tiles:group_elements_and_download_tiles:35 - Grouping elements by tile
2025-02-12 12:16:17.933 | INFO     | osm_ai_helper.group_elements_and_download_tiles:group_elements_and_download_tiles:40 - Downloading tiles and writing annotation
2025-02-12 12:16:17.935 | INFO     | osm_ai_helper.group_elements_and_download_tiles:group_elements_and_download_tiles:43 - Processed 0/11 tiles


In [11]:
from osm_ai_helper.convert_to_yolo_dataset import convert_to_yolo_dataset

In [12]:
convert_to_yolo_dataset(f"datasets/{TRAIN_AREA}")

In [13]:
convert_to_yolo_dataset(f"datasets/{TEST_AREA}")

# Check out of the box predictions

In [ ]:
from pathlib import Path
from ultralytics import YOLO

In [ ]:
yolo = YOLO("yolo11m.pt")

In [ ]:
yolo.predict(list(Path(f"datasets/{TEST_AREA}").glob("*.jpg"))[0], save=True)

# Upload Dataset

In [ ]:
Path("yolo_dataset.yaml").write_text(
    f"""
path: .
train: {TRAIN_AREA}
val: {TEST_AREA}

names:
  0: {CLASS_NAME}
"""
)

68

100%|██████████| 38.8M/38.8M [00:00<00:00, 291MB/s]
